# 4.8 Grouping Data & Aggregating Variables

### Import Libraries: Pandas, Numpy, Os

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import time

start_time = time.time()

### Create path to 'Prepared data'

In [2]:
# Create path to Prepared data folder
path_prepared = r'C:\Users\danie\Desktop\career Foundry\31-03-2023 Instacart Basket Analysis\02 Data\Prepared data'

### Import DataFrame, Pickle file (.pkl)

In [3]:
# Import .pkl
df_ords_prods = pd.read_pickle(os.path.join(path_prepared, '4.8.orders_products_merged.pkl'))

### Check columns & shape

In [4]:
# Check dataframe with '.head()'
df_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,Monday,8,0.0,196,1,0,Soda,77,7,9.0,Min-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,Tuesday,7,15.0,196,1,1,Soda,77,7,9.0,Min-range product,Regularly busy,Slowest days,Average orders,10,New customer
2,473747,1,3,Tuesday,12,21.0,196,1,1,Soda,77,7,9.0,Min-range product,Regularly busy,Slowest days,Most orders,10,New customer
3,2254736,1,4,Wednesday,7,29.0,196,1,1,Soda,77,7,9.0,Min-range product,Least busy,Slowest days,Average orders,10,New customer
4,431534,1,5,Wednesday,15,28.0,196,1,1,Soda,77,7,9.0,Min-range product,Least busy,Slowest days,Most orders,10,New customer


In [5]:
# Check dataframe shape
df_ords_prods.shape

(32404859, 19)

***The shape of the file imported match with the shape of the file previously exported in the last exercise.***

## 02. 'order_number' group by 'departmet_id'

In [6]:
# Group 'order_number' by 'department_id' adding the 'order_number' mean for each 'department 
df_ords_prods.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


***Q.3. We observe 21 department_ids and the max and min in the order_number's mean had change significantly. Previusly, using the first million entries only 8 out of 21 department was reflected in the data frame,therefore, we can assume that no order from the others 13 department was made or registered within the first million rows of our data set.***

## 0.4 Create 'loyalty_flag'

In [7]:
# Split the data in to groups base on 'user_id'
# Apply transform() function on 'order_number'
# Create 'max_order' column to place the results of the aggregation
df_ords_prods['max_orders'] = df_ords_prods.groupby(['user_id'])['order_number'].transform(np.max)

In [8]:
# Assign a "loyalty" label to a user ID
df_ords_prods.loc[df_ords_prods['max_orders'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
# Assign a "loyalty" label to a user ID
df_ords_prods.loc[(df_ords_prods['max_orders'] <= 40) & (df_ords_prods['max_order'] > 10), 'loyalty_flag'] = 'Regular customers'

In [10]:
#Assign a "loyalty" label to a user ID
df_ords_prods.loc[df_ords_prods['max_orders'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Check 'Loyalty_flag' frequency
df_ords_prods['loyalty_flag'].value_counts(dropna = False)

Regular customers    15876776
Loyal customer       10284093
New customer          6243990
Name: loyalty_flag, dtype: int64

## 05. Determine if whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [12]:
# Summary check "loyalty_flag" using groupby() & agg() functions
df_ords_prods.groupby('loyalty_flag').agg({'prices': ['mean', 'median', 'max', 'min', 'sum']})

prices                                  
                        mean median      max  min          sum
loyalty_flag                                                  
Loyal customer     10.386336    7.4  99999.0  1.0  106814042.2
New customer       13.294670    7.4  99999.0  1.0   83011787.2
Regular customers  12.495717    7.4  99999.0  1.0  198391693.2

***Before any further analysis, I observe outliers values in 'prices' variable values which could be a error imputing the data or a mayor transaction that can influence signifycantly the results therefore would not represent the general user behaviour. The identification and omision of these values for further analysis will be conducted.***

### Detecting and deleting unusuals values in 'prices' column 

In [13]:
# Locate the upper outliers in 'prices' column
df_ords_prods.groupby('prices').agg({'product_id': ['min']})

,product_id
,min
prices,
1.0,236
1.1,8
1.2,137
1.3,36
1.4,260
...,...
24.8,9302
24.9,9896


***There are two values, '14900' & '99999' in 'prices' that should be categorized as 'outliers'; therefore, both observations will be omitted in the analysis***

***I suspect that the Product_id 21553 with priced 14900.0 was wrongly imputted, and I assume it should be '14.9'. It must be verify before adding this entries in our studies.***

In [14]:
# Delete entries (rows) with values'14900' & '99999' in column 'prices' 
df_ords_prods = df_ords_prods.drop(df_ords_prods[df_ords_prods.product_id == 21553].index)

In [15]:
df_ords_prods = df_ords_prods.drop(df_ords_prods[df_ords_prods.product_id == 33664].index)

In [16]:
# Check Data set after deleting rows 
df_ords_prods.groupby('prices').agg({'product_id': ['min']})

,product_id
,min
prices,
1.0,236
1.1,8
1.2,137
1.3,36
1.4,260
...,...
24.6,8441
24.7,10814


***Both product_ids succefuly remove without compromise the integrity of the data set.---Total of 5127 observations out of 32404859 removed it represent 0.00015% of entries in the data set.***

In [17]:
# Statistical summary using agg() function
df_ords_prods.groupby('loyalty_flag').agg({'prices': ['mean', 'median', 'max', 'min', 'sum', 'count']})

prices                                         
                       mean median   max  min          sum     count
loyalty_flag                                                        
Loyal customer     7.773575    7.4  25.0  1.0   79933825.2  10282763
New customer       7.801206    7.4  25.0  1.0   48701689.2   6242841
Regular customers  7.798262    7.4  25.0  1.0  123790606.2  15874128

***There is a small difference in Average spent between all three groups.
    ---Loyal customer average spend: 7.773575.
    ---New customer average spend: 7.801206.
    ---Reagular customers 7.798262.***

***All groups share the same values in Median = 7.4, Maximum = 25.0, Minimum = 1.0***

***The bigger group based in "loyalty_flag" is "Regular customers": 15874128 users. Follow by "Loyal customer": 10282763 users and "New customer": 6242841 users*** 

## 06. Create 'Spending_flag' with prices average.

In [18]:
# Create new column with the average spend by user_id using groupby() & agg() functions. 
# Transform(np.mean) to get the 'prices' average.
df_ords_prods['average_spend'] = df_ords_prods.groupby(['user_id'])['prices'].transform(np.mean)

In [19]:
df_ords_prods['average_spend'].value_counts(dropna = False).head(7)

8.00    10484
7.50     9116
8.10     8366
7.75     7432
7.40     7044
7.70     6467
7.30     6391
Name: average_spend, dtype: int64

In [22]:
# Creating new column 'spender_flag' to categorize the users in two groups base on 'average_spend'
df_ords_prods.loc[(df_ords_prods['average_spend'] < 10), 'spender_flag'] = 'Low spender'

df_ords_prods.loc[(df_ords_prods['average_spend'] >= 10), 'spender_flag'] = 'High spender'

# Frequency based on 'spander_flag
df_ords_prods.groupby('spender_flag').agg({'user_id': ['count']})

,user_id
,count
spender_flag,
High spender,119719
Low spender,32280013


***Number of users categorized as 'High spender': 119719***

***Number of users categorized as 'Low spender': 32280013***

In [23]:
df_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,max_orders,average_spend,spender_flag
0,2539329,1,1,Monday,8,0.0,196,1,0,Soda,...,9.0,Min-range product,Regularly busy,Regularly busy,Average orders,10,New customer,10,6.367797,Low spender
1,2398795,1,2,Tuesday,7,15.0,196,1,1,Soda,...,9.0,Min-range product,Regularly busy,Slowest days,Average orders,10,New customer,10,6.367797,Low spender
2,473747,1,3,Tuesday,12,21.0,196,1,1,Soda,...,9.0,Min-range product,Regularly busy,Slowest days,Most orders,10,New customer,10,6.367797,Low spender
3,2254736,1,4,Wednesday,7,29.0,196,1,1,Soda,...,9.0,Min-range product,Least busy,Slowest days,Average orders,10,New customer,10,6.367797,Low spender
4,431534,1,5,Wednesday,15,28.0,196,1,1,Soda,...,9.0,Min-range product,Least busy,Slowest days,Most orders,10,New customer,10,6.367797,Low spender


## 07. Create frequency flag based on 'days_since_prior_order'

In [24]:
# Creating a new column 'user_median' with the median values base on 'days_since_prior_order' for a new "column flag" 
df_ords_prods['user_median'] = df_ords_prods.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [25]:
df_ords_prods['user_median'].head()

0    20.0
1    20.0
2    20.0
3    20.0
4    20.0
Name: user_median, dtype: float64

In [26]:
# Creating users 'frequency_flag' column 
df_ords_prods.loc[(df_ords_prods['user_median'] > 20), 'frequency_flag'] = 'Non-frequent customer'

In [27]:
df_ords_prods.loc[(df_ords_prods['user_median'] > 10) & (df_ords_prods['user_median'] <= 20), 'frequency_flag'] = 'Regular customer'

In [28]:
df_ords_prods.loc[(df_ords_prods['user_median'] < 20), 'frequency_flag'] = 'Frequent customer'

In [29]:
df_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,max_orders,average_spend,spender_flag,user_median,frequency_flag
0,2539329,1,1,Monday,8,0.0,196,1,0,Soda,...,Regularly busy,Regularly busy,Average orders,10,New customer,10,6.367797,Low spender,20.0,Regular customer
1,2398795,1,2,Tuesday,7,15.0,196,1,1,Soda,...,Regularly busy,Slowest days,Average orders,10,New customer,10,6.367797,Low spender,20.0,Regular customer
2,473747,1,3,Tuesday,12,21.0,196,1,1,Soda,...,Regularly busy,Slowest days,Most orders,10,New customer,10,6.367797,Low spender,20.0,Regular customer
3,2254736,1,4,Wednesday,7,29.0,196,1,1,Soda,...,Least busy,Slowest days,Average orders,10,New customer,10,6.367797,Low spender,20.0,Regular customer
4,431534,1,5,Wednesday,15,28.0,196,1,1,Soda,...,Least busy,Slowest days,Most orders,10,New customer,10,6.367797,Low spender,20.0,Regular customer


In [30]:
# Check frequency on 'frequency_flag' column 
df_ords_prods.groupby('frequency_flag').agg({'user_id': ['count']})

,user_id
,count
frequency_flag,
Frequent customer,29387250
Non-frequent customer,2685973
Regular customer,326509


In [31]:
print(time.time() - start_time)

57.26626205444336


In [32]:
#Dropping duplicated column
df_ords_prods = df_ords_prods.drop(columns = ['max_order'])

In [35]:
df_ords_prods.shape


(32399732, 23)

In [36]:
#Exporting data set in .pkl format
start_time = time.time()

df_ords_prods.to_pickle(os.path.join(path_prepared, '4.8_orders_products_merged.pkl'))

print(time.time() - start_time)

17.960956811904907
